In [3]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import optuna

In [4]:
def make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight):
    """
    Crea e restituisce un'istanza dell'ambiente "HalfCheetah-v5" dalla libreria Gymnasium con parametri personalizzati.

    Args:
        reset_noise_scale (float): Intensità del rumore all'inizio di ogni episodio.
        forward_reward_weight (float): Peso della ricompensa per il movimento in avanti.
        ctrl_cost_weight (float): Peso del costo del controllo (energia usata per muoversi).

    Returns:
        gym.Env: Un'istanza dell'ambiente con i parametri specificati.
    """
    return gym.make("HalfCheetah-v5",
                    reset_noise_scale=reset_noise_scale,
                    forward_reward_weight=forward_reward_weight,
                    ctrl_cost_weight=ctrl_cost_weight)


def objective(trial):
    """
    Funzione obiettivo per Optuna, che definisce il processo di tuning degli iperparametri per PPO.

    Optuna esplora diverse combinazioni di parametri per trovare la configurazione che massimizza
    la ricompensa media dell'agente.

    Args:
        trial (optuna.Trial): Oggetto che permette di campionare nuovi iperparametri.

    Returns:
        float: Ricompensa media ottenuta dopo l'addestramento con i parametri selezionati.
    """
    # Ricerca dei migliori parametri dell'ambiente tramite Optuna
    reset_noise_scale = trial.suggest_float('reset_noise_scale', 0.005, 0.3)  # Intensità del rumore all'inizio
    forward_reward_weight = trial.suggest_float('forward_reward_weight', 0.7, 1.5)  # Peso della ricompensa
    ctrl_cost_weight = trial.suggest_float('ctrl_cost_weight', 0.02, 0.3)  # Peso del costo del controllo

    # Creazione dell'ambiente con i parametri suggeriti
    env = make_env(reset_noise_scale, forward_reward_weight, ctrl_cost_weight)
    env = DummyVecEnv([lambda: env])  # Necessario per Stable-Baselines3
    env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)  # Normalizzazione delle osservazioni e ricompense

    # Ricerca degli iperparametri per l'algoritmo PPO
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)  # Range logaritmico per migliore esplorazione
    n_steps = trial.suggest_int('n_steps', 1024, 16384, step=1024)  # Numero di passi prima dell'aggiornamento
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256, 512, 1024])  # Dimensione del batch
    gamma = trial.suggest_float('gamma', 0.97, 0.9999)  # Fattore di sconto per le ricompense future
    gae_lambda = trial.suggest_float('gae_lambda', 0.85, 1.0)  # Trade-off tra bias e varianza nel GAE
    clip_range = trial.suggest_float('clip_range', 0.1, 0.35)  # Clip per la perdita PPO
    ent_coef = trial.suggest_float('ent_coef', 1e-5, 0.05)  # Coefficiente di entropia per regolare l'esplorazione

    # Creazione e addestramento del modello PPO con i parametri scelti
    model = PPO("MlpPolicy", env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                batch_size=batch_size,
                gamma=gamma,
                gae_lambda=gae_lambda,
                clip_range=clip_range,
                ent_coef=ent_coef,
                verbose=0)  # Disabilita output per Optuna
    model.learn(total_timesteps=150000)  # Addestramento del modello

    # Disattiviamo l'aggiornamento della normalizzazione per la valutazione finale
    env.training = False
    env.norm_reward = False

    # Valutiamo il modello su 100 episodi
    mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=100, deterministic=True)

    print(f'Mean reward: {mean_reward}')
    return mean_reward  # Optuna massimizzerà questa ricompensa


# Creazione dello studio Optuna e avvio dell'ottimizzazione
study = optuna.create_study(direction='maximize')  # Massimizziamo la ricompensa media
study.optimize(objective, n_trials=100)  # Eseguiamo 100 tentativi con Optuna

# Stampa dei migliori iperparametri trovati
print("Best hyperparameters: ", study.best_params)



[I 2025-02-20 19:45:15,239] A new study created in memory with name: no-name-1892ba40-77bc-4286-95f5-30555f36ed5a
/var/folders/d5/8f8lxk3x1gddrb19zv8xl57m0000gn/T/ipykernel_43646/2993520663.py:23: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)  # Range più ampio
[I 2025-02-20 19:46:02,765] Trial 0 finished with value: -169.96075044236377 and parameters: {'reset_noise_scale': 0.005407723526896945, 'forward_reward_weight': 1.265691657276868, 'ctrl_cost_weight': 0.10597432710663796, 'learning_rate': 3.2392866668478713e-05, 'n_steps': 14336, 'batch_size': 512, 'gamma': 0.9967491942949904, 'gae_lambda': 0.8770345379385267, 'clip_range': 0.16007754591201798, 'ent_coef': 0.009780012810592625}. Best is trial 0 with value: -169.96075044236377.


Mean reward: -169.96075044236377


[I 2025-02-20 19:46:48,810] Trial 1 finished with value: 728.330663225758 and parameters: {'reset_noise_scale': 0.0666162714817425, 'forward_reward_weight': 1.3745629927321112, 'ctrl_cost_weight': 0.1134428355179037, 'learning_rate': 0.0003689745029262415, 'n_steps': 5120, 'batch_size': 512, 'gamma': 0.9969248931970972, 'gae_lambda': 0.8713390232527614, 'clip_range': 0.2519626812357027, 'ent_coef': 0.012319628051307801}. Best is trial 1 with value: 728.330663225758.


Mean reward: 728.330663225758


[I 2025-02-20 19:47:42,259] Trial 2 finished with value: 1055.501501187583 and parameters: {'reset_noise_scale': 0.14229291483494422, 'forward_reward_weight': 0.8155771709086337, 'ctrl_cost_weight': 0.033211708592419266, 'learning_rate': 0.0001147387479895581, 'n_steps': 1024, 'batch_size': 128, 'gamma': 0.9849348980072848, 'gae_lambda': 0.8932729333364544, 'clip_range': 0.3016255849216237, 'ent_coef': 0.011696879105870743}. Best is trial 2 with value: 1055.501501187583.


...

Mean reward: 2120.0764839226413


[I 2025-02-20 21:13:05,461] Trial 98 finished with value: 3553.0941224342237 and parameters: {'reset_noise_scale': 0.04143153804195014, 'forward_reward_weight': 1.3741765081158461, 'ctrl_cost_weight': 0.06965838903486682, 'learning_rate': 0.0002869089598817114, 'n_steps': 1024, 'batch_size': 128, 'gamma': 0.9964383155408642, 'gae_lambda': 0.9443600164220389, 'clip_range': 0.17616056689611564, 'ent_coef': 0.005113659605647694}. Best is trial 60 with value: 4698.895084980981.


Mean reward: 3553.0941224342237


[I 2025-02-20 21:13:59,065] Trial 99 finished with value: 1864.2691478518284 and parameters: {'reset_noise_scale': 0.023340401689453526, 'forward_reward_weight': 1.3703010923961716, 'ctrl_cost_weight': 0.06955308847223407, 'learning_rate': 0.00026606278983739364, 'n_steps': 1024, 'batch_size': 128, 'gamma': 0.996102404541061, 'gae_lambda': 0.9671458744694584, 'clip_range': 0.16987597676513275, 'ent_coef': 0.004711116174813325}. Best is trial 60 with value: 4698.895084980981.


Mean reward: 1864.2691478518284
Best hyperparameters:  {'reset_noise_scale': 0.06369684242560415, 'forward_reward_weight': 1.44617835103446, 'ctrl_cost_weight': 0.0544226645937837, 'learning_rate': 0.0009699365825096747, 'n_steps': 1024, 'batch_size': 1024, 'gamma': 0.9857859792010212, 'gae_lambda': 0.9107811154406141, 'clip_range': 0.2091038335716418, 'ent_coef': 0.0004353350261388343}
